In [1]:
import wandb

In [2]:
api = wandb.Api()
project="subset-search-gpu-opt"
entity="johnny-gary"
runs = api.runs(entity + "/" + project)

In [3]:
runs

<Runs johnny-gary/subset-search-gpu-opt>

In [6]:
import pandas as pd

In [20]:
summary_list, tags_list, name_list = [], [], []
for run in runs:
    summary_list.append(run.summary._json_dict)
    name_list.append(run.name)
    tags_list.append(run.tags)


runs_df = pd.DataFrame({
    "SUMMARY": summary_list,
    "NAME": name_list, 
    "TAGS": tags_list
    })


In [21]:
runs_df

,SUMMARY,NAME,TAGS
0,"{'_timestamp': 1665376228, 'batch_size': 8, 's...",feasible-paper-64,"[44, correlation, roberta-large, train_size-100]"
1,"{'loss': 0.0024962446186691523, '_step': 2110,...",hearty-elevator-63,"[43, correlation, google/electra-small-discrim..."
2,"{'sst:val_acc': 0.5422343324250681, 'sst2_fina...",spring-salad-62,"[43, correlation, roberta-large, train_size-100]"
3,"{'_timestamp': 1665375471, 'batch_size': 8, 's...",rich-sea-61,"[42, correlation, google/electra-small-discrim..."
4,"{'_wandb': {'runtime': 429}, '_timestamp': 166...",vague-haze-60,"[42, correlation, roberta-large, train_size-100]"
...,...,...,...
58,"{'_timestamp': 1664077525, 'sst:val_acc': 0.68...",usual-bird-5,[small_batch_8]
59,"{'_wandb': {'runtime': 145}, '_runtime': 146, ...",sunny-surf-4,[large_batch_32]
60,"{'_step': 1808, '_runtime': 164, '_timestamp':...",smart-plasma-3,[small_batch_8]
61,"{'loss': 0.0001908041158458218, '_runtime': 17...",efficient-haze-2,[large_batch_32]


In [12]:
runs_df.iloc[1].SUMMARY

{'loss': 0.0024962446186691523,
 '_step': 2110,
 'run_time': 176.5,
 '_timestamp': 1665376087,
 '_wandb': {'runtime': 178},
 '_runtime': 179,
 'batch_size': 8,
 'sst:val_acc': 0.6457765667574932,
 'sst2_final_test:accuracy': 0.6248868778280543,
 'sst2_final_valid:accuracy': 0.6457765667574932}